In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys
import os
import matplotlib.image as mpimg
import pandas as pd
from keras import backend as keras
from keras.utils import plot_model
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
def Corr(x, max_disp=40):
    x1 = x[0]
    x2 = x[1]
    w = tf.shape(x2)[2]
    corr_tensors = []
    for i in range(-max_disp, 0, 1):
        shifted = tf.pad(x2[:, :, 0:w+i, :], [[0, 0], [0, 0], [-i, 0], [0, 0]], "CONSTANT")
        corr = tf.reduce_mean(tf.multiply(shifted, x1), axis=3)
        corr_tensors.append(corr)
    for i in range(max_disp + 1):
        shifted = tf.pad(x1[:, :, i:, :], [[0, 0], [0, 0], [0, i], [0, 0]], "CONSTANT")
        corr = tf.reduce_mean(tf.multiply(shifted, x2), axis=3)
        corr_tensors.append(corr)
    return tf.transpose(tf.stack(corr_tensors), perm=[1, 2, 3, 0])

## Model

In [4]:
def iresnet_ver1(shape=(None,None,3)):
    
    activation_relu = 'relu'
    init_random_normal = 'random_normal'
    
    # Input Layer
    
    input_left = Input(shape)
    input_right = Input(shape)
    
    #Stem Block for multiscale shared feature extraction begins
    
    conv1 = Conv2D(64, 7, strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name='conv1')
    conv1_left = conv1(input_left)
    conv1_right = conv1(input_right)
    
    up1 = Conv2DTranspose(32, 4, strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name='up1')
    up1_left = up1(conv1_left)
    up1_right = up1(conv1_right)
    
    conv2 = Conv2D(128, 5, strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name='conv2')
    conv2_left = conv2(conv1_left)
    conv2_right = conv2(conv1_right)
    
    up2 = Conv2DTranspose(32, 8, strides=4, activation=activation_relu, kernel_initializer=init_random_normal, name='up2')
    up2_left = up2(conv2_left)
    up2_right = up2(conv2_right)
    
    match_conv1 = Conv2D(32, 5, strides=1, activation=activation_relu,kernel_initializer=init_random_normal, name='matchconv1')
    match_conv1_left = match_conv1(up1_left)
    match_conv1_right = match_conv1(up1_right)
    
    up1_2_left = concatenate([match_conv1_left,up2_left],name='concat_left')
    up1_2_right = concatenate([match_conv1_right,up2_right],name='conact_right')
    
    conv1_2 = Conv2D(32, 1, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='convup1_2')
    conv1_2_left = conv1_2(up1_2_left)
    conv1_2_right = conv1_2(up1_2_right)
    
    #Stem Block for multiscale shared feature extraction ends
    
    model_shared_features = Model(inputs=[input_left,input_right],outputs=[conv1_2_left,conv1_2_right])
    model_shared_features.compile(optimizer = Adam(lr = 0.00001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model_shared_features.summary()
    plot_model(model=model_shared_features,show_layer_names=True, show_shapes=True, to_file='model_shared_features_v1.png')
    
    #Initial Disparity Estimation Sub-network begins

    corr1d = Lambda(Corr,arguments={'max_disp':40}, name='corr1d')([conv2_left,conv2_right])
    conv_redir = Conv2D(64, 1, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='conv_redir')(conv2_left)
    
    corr1d_conv_redir = concatenate([corr1d,conv_redir],name='concat_corr1d_convredir')
    conv3 = Conv2D(256, 3, strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name='conv3')(corr1d_conv_redir)
    conv3 = Conv2D(256, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='conv3_1')(conv3)
    
    conv4 = Conv2D(512, 3, strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name='conv4')(conv3)
    conv4= Conv2D(512, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='conv4_1')(conv4)
    
    conv5 = Conv2D(512, 3, strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name='conv5')(conv4)
    conv5 = Conv2D(512, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='conv5_1')(conv5)
    
    conv6 = Conv2D(1024, 3, strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name='conv6')(conv5)
    conv6 = Conv2D(1024, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='conv6_1')(conv6)
    disp6 = Conv2D(1, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='disp6')(conv6)
    
    matchuconv5_disp6 = Conv2DTranspose(1,8,strides=2,activation=activation_relu, kernel_initializer=init_random_normal,name='matchuconv5_disp6')(disp6)
    matchconv5_conv5_1 = Conv2D(512, 4, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv5_conv5_1')(conv5)
    
    unconv5 = Conv2DTranspose(512,4,strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name= 'unconv5')(conv6)
    iconv5_concat = concatenate([unconv5,matchuconv5_disp6,matchconv5_conv5_1], name='iconv5_concat')
    iconv5 = Conv2D(512, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='iconv5')(iconv5_concat)
    disp5 = Conv2D(1, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='disp5')(iconv5)
    
    match_disp5 = Conv2DTranspose(1,8,strides=2,activation=activation_relu, kernel_initializer=init_random_normal,name='matchuconv4_disp5')(disp5)
    
    match_conv4 = Conv2D(512, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv4_1_conv4_1')(conv4)
    match_conv4 = MaxPool2D(pool_size=2, strides=1,name='matchpool4_1_conv4_1')(match_conv4)
    match_conv4 = Conv2D(512, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv4_2_conv4_1')(match_conv4)
    match_conv4 = MaxPool2D(pool_size=2, strides=1,name='matchpool4_2_conv4_1')(match_conv4)
    match_conv4 = Conv2D(512, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv4_3_conv4_1')(match_conv4)
    match_conv4 = MaxPool2D(pool_size=2, strides=1,name='matchpool4_3_conv4_1')(match_conv4)
    match_conv4 = Conv2D(512, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv4_4_conv4_1')(match_conv4)
    match_conv4 = MaxPool2D(pool_size=3, strides=1,name='matchpool4_4_conv4_1')(match_conv4)
    match_conv4 = Cropping2D(cropping=((0,0),(0,1)),name="matchcrop4_conv4_1")(match_conv4) 
    
    unconv4 = Conv2DTranspose(256,4,strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name= 'unconv4')(iconv5)
    iconv4_concat = concatenate([unconv4,match_disp5,match_conv4], name='iconv4_concat')
    iconv4 = Conv2D(512, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='iconv4')(iconv4_concat)
    disp4 = Conv2D(1, 3, strides=1, activation=activation_relu,kernel_initializer=init_random_normal, name='disp4')(iconv4)
    
    match_disp4 = Conv2DTranspose(1,8,strides=2,activation=activation_relu, kernel_initializer=init_random_normal,name='matchupconv3_disp4')(disp4)
    
    match_conv3 = Conv2D(256, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv3_1_conv3_1')(conv3)
    match_conv3 = MaxPool2D(pool_size=3, strides=1,name='matchpool3_1_conv3_1')(match_conv3)
    match_conv3 = Conv2D(256, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv3_2_conv3_1')(match_conv3)
    match_conv3 = MaxPool2D(pool_size=3, strides=1,name='matchpool3_2_conv3_1')(match_conv3)
    match_conv3 = Conv2D(256, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv3_3_conv3_1')(match_conv3)
    match_conv3 = MaxPool2D(pool_size=3, strides=1,name='matchpool3_3_conv3_1')(match_conv3)
    match_conv3 = Conv2D(256, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv3_4_conv3_1')(match_conv3)
    match_conv3 = MaxPool2D(pool_size=3, strides=1,name='matchpool3_4_conv3_1')(match_conv3)
    match_conv3 = Conv2D(256, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv3_5_conv3_1')(match_conv3)
    match_conv3 = MaxPool2D(pool_size=3, strides=1,name='matchpool3_5_conv3_1')(match_conv3)
    match_conv3 = Conv2D(256, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv3_6_conv3_1')(match_conv3)
    match_conv3 = MaxPool2D(pool_size=3, strides=1,name='matchpool3_6_conv3_1')(match_conv3)
    match_conv3 = Cropping2D(cropping=((0,0),(1,1)),name="matchcrop3_conv3_1")(match_conv3)
    
    unconv3 = Conv2DTranspose(128,4,strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name= 'unconv3')(iconv4)
    iconv3_concat = concatenate([unconv3,match_disp4,match_conv3], name='iconv3_concat')
    iconv3 = Conv2D(128, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='iconv3')(iconv3_concat)
    disp3 = Conv2D(1, 3, strides=1, activation=activation_relu,kernel_initializer=init_random_normal, name='disp3')(iconv3)
    
    match_disp3 = Conv2DTranspose(1,8,strides=2,activation=activation_relu, kernel_initializer=init_random_normal,name='match_disp3')(disp3)
    
    match_conv2 = Conv2D(128, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_1_conv2_1')(conv2_left)
    match_conv2 = Conv2D(128, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_2_conv2_1')(match_conv2)
    match_conv2 = MaxPool2D(pool_size=3, strides=1,name='matchpool2_1_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_3_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_4_conv2_1')(match_conv2)
    match_conv2 = MaxPool2D(pool_size=3, strides=1,name='matchpool2_2_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_5_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_6_conv2_1')(match_conv2)
    match_conv2 = MaxPool2D(pool_size=3, strides=1,name='matchpool2_3_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_7_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_8_conv2_1')(match_conv2)
    match_conv2 = MaxPool2D(pool_size=3, strides=1,name='matchpool2_4_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_9_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_10_conv2_1')(match_conv2)
    match_conv2 = MaxPool2D(pool_size=3, strides=1,name='matchpool2_5_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_11_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_12_conv2_1')(match_conv2)
    match_conv2 = MaxPool2D(pool_size=3, strides=1,name='matchpool2_6_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_13_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_14_conv2_1')(match_conv2)
    match_conv2 = MaxPool2D(pool_size=3, strides=1,name='matchpool2_7_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_15_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_16_conv2_1')(match_conv2)
    match_conv2 = MaxPool2D(pool_size=3, strides=1,name='matchpool2_8_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_17_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_18_conv2_1')(match_conv2)
    match_conv2 = MaxPool2D(pool_size=3, strides=1,name='matchpool2_9_conv2_1')(match_conv2)
    match_conv2 = Conv2D(128, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv2_19_conv2_1')(match_conv2)
    match_conv2 = MaxPool2D(pool_size=2, strides=1,name='matchpool2_10_conv2_1')(match_conv2)
    match_conv2 = Cropping2D(cropping=((0,0),(2,2)),name="matchcrop2_conv2_1")(match_conv2)
    
    unconv2 = Conv2DTranspose(64,4,strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name= 'unconv2')(iconv3)
    iconv2_concat = concatenate([unconv2,match_disp3,match_conv2], name='iconv2_concat')
    iconv2 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='iconv2')(iconv2_concat)
    disp2 = Conv2D(1, 3, strides=1, activation=activation_relu,kernel_initializer=init_random_normal, name='disp2')(iconv2)
    
    match_disp2 = Conv2DTranspose(1,8,strides=2,activation=activation_relu, kernel_initializer=init_random_normal,name='match_disp2')(disp2)
    
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_1_conv1_1')(conv1_left)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_2_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_1_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_3_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_4_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_2_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_5_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_6_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_3_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_7_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_8_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_4_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_9_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_10_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_5_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_11_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_12_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_6_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_13_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_14_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_7_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_15_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_16_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_8_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_17_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_18_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_9_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_19_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_20_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_10_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_21_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_22_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_11_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_23_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_24_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_12_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_25_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_26_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_13_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_27_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_28_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_14_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_29_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_30_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_15_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_31_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_32_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_16_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_33_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_34_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_17_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_35_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_36_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_18_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_37_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_38_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=3, strides=1,name='matchpool1_19_conv1_1')(match_conv1)
    match_conv1 = Conv2D(64, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv1_39_conv1_1')(match_conv1)
    match_conv1 = MaxPool2D(pool_size=2, strides=1,name='matchpool1_20_conv1_1')(match_conv1)
    match_conv1 = Cropping2D(cropping=((0,0),(4,4)),name="matchcrop1_conv1_1")(match_conv1)
    
    unconv1 = Conv2DTranspose(64,4,strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name= 'unconv1')(iconv2)
    iconv1_concat = concatenate([unconv1,match_disp2,match_conv1], name='iconv1_concat')
    iconv1 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='iconv1')(iconv1_concat)
    disp1 = Conv2D(1, 3, strides=1, activation=activation_relu,kernel_initializer=init_random_normal, name='disp1')(iconv1)
    
    match_disp1 = Conv2DTranspose(1,8,strides=2,activation=activation_relu, kernel_initializer=init_random_normal,name='match_disp1')(disp1)
    
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_1_up1_2')(conv1_2_left)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_1_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_1_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_2_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_2_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_2_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_3_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_3_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_3_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_4_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_4_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_4_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_5_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_5_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_5_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_6_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_6_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_6_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_7_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_7_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_7_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_8_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_8_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_8_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_9_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_9_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_9_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_10_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_10_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_10_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_11_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_11_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_11_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_12_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_12_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_12_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_13_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_13_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_13_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_14_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_14_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_14_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_15_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_15_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_15_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_16_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_16_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_16_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_17_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_17_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_17_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_18_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_18_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_18_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_19_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_19_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_19_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_20_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_20_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_20_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_21_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_21_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_21_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_22_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_22_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_22_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_23_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_23_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_23_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_24_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_24_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_24_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_25_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_25_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_25_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_26_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_26_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_26_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_27_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_27_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_27_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_28_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_28_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_28_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_29_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_29_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_29_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_30_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_30_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_30_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_31_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_31_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_31_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_32_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_32_1_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_32_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_33_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_33_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_34_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_34_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_35_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_35_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_36_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_36_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_37_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_37_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_38_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_38_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_39_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_39_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_40_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_40_up1_2')(match_up1_2)
    match_up1_2 = Conv2D(32, 5, strides=1, activation=activation_relu, kernel_initializer=init_random_normal,name='matchconv0_41_up1_2')(match_up1_2)
    match_up1_2 = MaxPool2D(pool_size=3, strides=1,name='matchpool0_41_up1_2')(match_up1_2)
    match_up1_2 = Cropping2D(cropping=((0,0),(8,8)),name="matchcrop0_up1_2")(match_up1_2)
    
    unconv0 = Conv2DTranspose(32,4,strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name= 'unconv0')(iconv1)
    iconv0_concat = concatenate([unconv0,match_disp1,match_up1_2], name='iconv0_concat')
    iconv0 = Conv2D(32, 3, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='iconv0')(iconv0_concat)
    disp0 = Conv2D(1, 3, strides=1, activation=activation_relu,kernel_initializer=init_random_normal, name='disp0')(iconv0)
    
    #Initial Disparity Estimation Sub-network ends(DES-Net)
    
    model_init_disp = Model(inputs=[input_left,input_right],outputs=disp0)
    model_init_disp.compile(optimizer = Adam(lr = 0.00001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model_init_disp.summary()
    plot_model(model=model_init_disp,show_layer_names=True, show_shapes=True, to_file='model_init_disp_v1.png')
    
    #Disparity Refinement Sub-network begins(iRes-Net)
    
    
    
    return model_init_disp

In [5]:
x = iresnet_ver1((1280,720,3))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1280, 720, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1280, 720, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 637, 357, 64) 9472        input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
up1 (Conv2DTranspose)           (None, 1276, 716, 32 32800       conv1[0][0]                      
          